# Ml-Pipes

The objective of this project is to present a gentle introduction of how a machine learning model can be trained and deployed. And how [MLFlow](https://www.mlflow.org/), [FastAPI](https://fastapi.tiangolo.com/) and [Docker](https://docs.docker.com/) can facilite a couple aspects of such a task. Within this context, this notebook represents the development part of the model, where a data scientist would create a few models, evaluate it using some metrics and select the best one based in a metric. There are a lot of different machine learning tools that can be used in order to create models. It is important to note that the objective of this notebook is to give a broad overview of and end to end machine learning process, and therefore it is recommended to have the documentation of the frameworks used here as a companion. Also, the README file from the project explains how to reproduce the whole project.

The rest of the notebook is organized as follows
- 1) The Problem;
- 2) Setting up an MLFlow Experiment;
- 3) Training different Models;
- 4) Setting the Best Model to Production.

The code bellow imports everything that will be used throughout the notebook.

In [7]:
# Import Libs
import pandas as pd
pd.set_option('display.max_columns', 500)

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from settings import EXPERIMENT_NAME, FOLDS, CREDIT_CARD_MODEL_NAME,\
     CHAMPION_METRIC, THRESHOLD  # pylint: disable=import-error
from dao.CreditCardDefault \
    import load_creditcard_dataset  # pylint: disable=import-error
from trainers.h2o_automl import H2OClassifier  # pylint: disable=import-error
from trainers.pycaret import PycaretClassifier  # pylint: disable=import-error
from trainers.spark import SparkClassifier

## 1) The problem

The first thing we need to have to build a model is a problem to solve. Here it is used as example the [Credit Card Default from Kagle](https://www.kaggle.com/mlg-ulb/creditcardfraud), where basically the objective if to predict based on a few features whether or not a client will default on its credit card. The taret variable can assume the values 1, for default, and 0 for non default. Therefore it is a binary classification problem.

Bellow the dataset is imported and the first rows of the dataset. Note that the Time column has been removed from the original dataset.

In [2]:
dataset = load_creditcard_dataset()
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
55417,1.085133,-0.669989,0.822036,0.142537,-0.907347,0.363765,-0.715182,0.254263,1.354373,-0.499274,-1.158256,0.059617,-0.775425,-0.378222,0.083173,-0.221112,0.291495,-0.812330,0.281002,-0.023034,-0.211608,-0.488872,0.004122,-0.371560,0.079359,0.999670,-0.034705,0.014633,69.32,0
200517,1.092187,-1.936115,-0.742691,0.626054,-0.861825,0.735219,-0.367658,0.162170,0.974413,-0.035394,0.367724,1.024159,0.624077,-0.098820,0.121401,1.045670,-1.132820,0.805714,-0.025966,0.809354,0.309958,-0.037190,-0.129510,0.232609,-0.633778,0.095433,-0.084777,0.037491,462.51,0
129108,1.142044,0.109129,0.277380,1.205483,0.026661,0.321842,-0.072794,0.180119,0.126646,0.058191,0.786083,0.853191,-0.640728,0.339021,-0.717260,-0.408093,-0.078953,-0.316147,0.048457,-0.196701,-0.076447,-0.044667,-0.110400,-0.300581,0.660126,-0.308426,0.032219,0.001730,9.74,0
83676,-0.450198,1.181688,1.325129,0.358330,0.106443,-0.870807,0.827045,-0.367590,-0.152878,0.126591,-0.141445,-0.346248,-0.113214,-0.578416,1.276736,0.004538,0.131425,-0.046179,0.585897,0.213158,-0.227665,-0.473840,0.024050,0.332656,-0.730847,0.052158,-0.058601,0.016804,5.49,0
38963,1.277614,-0.050241,0.124556,0.118730,-0.155314,-0.084027,-0.177664,0.061263,0.304515,-0.047960,0.353242,0.527604,-0.462419,0.298590,-0.282083,0.370863,-0.612118,0.151401,0.851838,-0.098935,-0.259287,-0.686230,-0.048703,-0.504743,0.394324,0.483180,-0.051502,-0.009545,4.00,0


## 2) Setting up an MLFlow Experiment

Now that a problem has been stated and some data to help solving the problem has been gathered, the next step is to setup a MLFlow experiment to log our models. **MLFlow is built upon the concept of experiments. A experiment is a series of fits, where parameters, metrics, models and artifacts can be associated with the respective fit (in an machine learning package agnostic way).**

The code bellow tries to create an experiment, if that experiments already existis then it sets the experiment to the active one.
 

In [3]:
mlflow.set_tracking_uri("sqlite:///mlruns.db")
try:
    experiment = mlflow.create_experiment(EXPERIMENT_NAME)
except Exception:
    client = MlflowClient()
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri("sqlite:///mlruns.db")

2021/05/11 22:25:50 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2021/05/11 22:25:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

## 3) Training different Models

The ext step if to train, evaluate and log a few different models. In order to demonstrate that MLFlow allows us to use different machine learning packages we will train an H2O autoML and SkLearn models (using pycaret). Now is the time where MLFlow is put into action: For each model that if fitted it will be logged a few parameters, metrics, artifacts and the models it self. To understand how this is done it checkout the classifiers definitions in `src/trainers/` folder and the [MLFlow Logging Documentaion](https://www.mlflow.org/docs/latest/tracking.html#logging-data-to-runs), ot all happens inside the `mlflow.start_run()` context manager. 

The next cells will train different classifiers. Once they finish running you can deploy the [MLFlow Tracking UI](https://www.mlflow.org/docs/latest/tracking.html#tracking-ui) by executing `mlflow ui -p 5000 --backend-store-uri sqlite:///mlruns.db` in the terminal inside the `src/` folder. and see the results at [127.0.0.1:5000](127.0.0.1:5000).

In [4]:
H2OClassifier(
    run_name='H2O',
    max_mem_size='3G',
    threshold=THRESHOLD,
    df=dataset,
    target_col='Class',
    sort_metric='aucpr',
    max_models=8,
    max_runtime_secs=60,
    nfolds=FOLDS,
    seed=90
)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,47 mins 50 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_vinicius_crvatf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.894 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
AutoML progress: |
22:25:55.258: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Could not find exact threshold 0.5; using closest threshold found 0.48643484711647034.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] 

In [5]:
PycaretClassifier(
        experiment_name=EXPERIMENT_NAME,
        run_name='Pycaret',
        sort_metric='precision',
        df=dataset,
        target='Class',
        threshold=THRESHOLD,
        n_best_models=3,
        data_split_stratify=True,
        nfolds=FOLDS,
        normalize=True,
        transformation=True,
        ignore_low_variance=True,
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        session_id=54321
)

,Parameters
alpha,1.0
class_weight,None
copy_X,True
fit_intercept,True
max_iter,None
normalize,False
random_state,54321
solver,auto
tol,0.001


INFO  [logs] Visual Rendered Successfully
INFO  [logs] plot_model() succesfully completed......................................
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [10]:
SparkClassifier(
    df = dataset,
    target_col = 'Class',
    run_name = 'spark_classifier',
    max_mem_size = 4,
    n_cores = 4,
    seed = 90
)

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume

## 4) Setting the Best Model to Production

The final step in this notebook if to set to production the model with the best selected metric, imported as `CHAMPION_METRIC`. This is done to show is is possible to create an automated workflow using MLFlow to deplot a model. However it is also possible to deplot the model using the [UI server](https://www.mlflow.org/docs/latest/model-registry.html#ui-workflow).

Once this is done you can return to the README file to check how the model is now deployed.

In [6]:
# Getting The best Model according to CHAMPION_METRIC
champion = MlflowClient().search_runs(
    experiment_ids=[
        str(
            mlflow.get_experiment_by_name(name=EXPERIMENT_NAME).experiment_id
        )
    ],
    run_view_type=ViewType.ALL,
    order_by=[f"metrics.{CHAMPION_METRIC} DESC"],
    max_results=1
)
run_id = champion[0].info.run_id

# Registering Model in model registery
model = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=CREDIT_CARD_MODEL_NAME
)

# Setting version 1
MlflowClient().update_model_version(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    description='Deploying model with model registery'
)

# Setting it to production
MlflowClient().transition_model_version_stage(
    name=CREDIT_CARD_MODEL_NAME,
    version=model.version,
    stage="Production"
)

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'CreditCardDefault'.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2021/05/11 22:27:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CreditCardDefault, version 1
Created version '1' of model 'CreditCardDefault'.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [ale

<ModelVersion: creation_timestamp=1620782866842, current_stage='Production', description='Deploying model with model registery', last_updated_timestamp=1620782867409, name='CreditCardDefault', run_id='f5457e6e95814060833405d0cebeee75', run_link=None, source='./mlruns/1/f5457e6e95814060833405d0cebeee75/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>